In [5]:
import pandas as pd
df = pd.read_csv('demo_events.csv')
display(df.head(2))
df.shape

,Subject,Start date,Location,source,Start time,End time
0,Buntes Herbstfest im Tierpark,10/04/2024,"Tierpark Neumünster | 24537 Neumünster, Geerdt...",https://city-nms.de/events/veranstaltungskalen...,10:00,NaN
1,Die Spur des Bildhauers. Wiedersehen mit Heinz...,10/04/2024,"Herbert Gerisch-Stiftung | 24536 Neumünster, B...",https://city-nms.de/events/veranstaltungskalen...,12:00,18:00


(6, 6)

In [6]:
import os.path
from datetime import datetime, timedelta
import pandas as pd
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

from google.cloud import bigquery
from pandas_gbq import to_gbq

# Define the combined scopes
SCOPES = [
    "https://www.googleapis.com/auth/calendar",      # Google Calendar
    "https://www.googleapis.com/auth/bigquery"       # BigQuery
]

def authenticate_google_services():
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    # Return the Google Calendar service along with creds.
    service = build('calendar', 'v3', credentials=creds)
    return service, creds

service, creds = authenticate_google_services()  # Authenticate and get credentials for Google Calendar and BigQuery.

project_id = 'circular-maxim-436817-e3'
dataset_id = 'kierne35539432nun34b'
table_id = 'events'
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

client = bigquery.Client(credentials=creds, project=project_id)

# try:
#     client.delete_table(full_table_id, not_found_ok=True)  # Deleting existing "events" table.
#     print(f"""Table "{table_id}" deleted.""")              # This code can be deleted.
# except Exception as e:
#     print(f"An error occurred: {e}")

try:
    to_gbq(df, destination_table=full_table_id, project_id=project_id, if_exists='replace', credentials=creds, progress_bar=True)
    print("Data uploaded to BigQuery successfully!")   #  Inserting data into BigQuery "events" table.
except Exception as e:
    print(f"An error occurred: {e}")

Data uploaded to BigQuery successfully!


In [7]:
# Loading data from BigQuery back to python and then loading events into Google Calendar.
def create_event(service, event):
    """Creates an event in Google Calendar and returns the event ID."""
    event_result = service.events().insert(calendarId='primary', body=event).execute()
    print(f"Event created: {event_result['htmlLink']}")
    return event_result['id']

def load_events_from_csv():
    """Loads events from a CSV DataFrame and returns them as a list of event objects."""
    events = []

    query = f"SELECT * FROM `{full_table_id}`"
    df_loaded = client.query(query).to_dataframe()  # Load the data from BigQuery into a pandas DataFrame.
    print('The shape of loaded dataframe is:', df_loaded.shape)

    for _, row in df_loaded.iterrows():
        # Convert start date and time to datetime
        start_datetime_str = f"{row['Start date']}T{row['Start time']}"
        start_time_obj = pd.to_datetime(start_datetime_str)

        # Check if End time is provided; if not, add 27 minutes to Start time
        if pd.isna(row.get('End time')):  # If 'End time' is None or NaN
            end_time_obj = start_time_obj + timedelta(minutes=27)  # Add 27 minutes
        else:
            # Convert End time and assume same date as Start date
            end_datetime_str = f"{row['Start date']}T{row['End time']}"
            end_time_obj = pd.to_datetime(end_datetime_str)
        
        # Prepare the event details for Google Calendar
        event = {
            'summary': row['Subject'],
            'location': row['Location'],
            'start': {
                'dateTime': start_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
                'timeZone': 'Europe/Berlin',  # Set to your timezone
            },
            'end': {
                'dateTime': end_time_obj.strftime("%Y-%m-%dT%H:%M:%S"),
                'timeZone': 'Europe/Berlin',  # Set to your timezone
            }
        }
        events.append(event)
    return events

# Load the DataFrame containing event data (replace 'df_2' with the actual CSV DataFrame)
events = load_events_from_csv()

# Create events in Google Calendar and store event IDs
event_ids = []
for event in events:
    event_id = create_event(service, event)
    event_ids.append(event_id)

# Optionally save event IDs to a CSV for future reference
pd.DataFrame({'event_id': event_ids}).to_csv('event_ids.csv', index=False)

The shape of loaded dataframe is: (6, 6)
Event created: https://www.google.com/calendar/event?eid=M2M3N2wzaWNxdmdlb2ZxNTQ5ZDh0NW43NzggZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=OGZsazFocmU3YnJiN2ZyMXZpbnQybmhmcm8gZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=dm1ncm41N3NhY29idG5rdHF1MzZzYzd1ZDggZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=MXUyNjY4YmoxcXJqZ3VvNzljZGF0NnM0cGcgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=bXNnc3VkanEwdXNvODJtdTV0OHBlYmNqcWcgZm9rczI0M2RhYmw0QG0
Event created: https://www.google.com/calendar/event?eid=MTBscXI3MmcxajdxdTVucTk1c3JrOG1nZDQgZm9rczI0M2RhYmw0QG0


In [9]:
def delete_all_events(service, event_ids):
    """Deletes all events from Google Calendar using a list of event IDs."""
    for event_id in event_ids:
        try:
            service.events().delete(calendarId='primary', eventId=event_id).execute()
            print(f"Event with ID {event_id} deleted successfully.")
        except Exception as e:
            print(f"Error deleting event {event_id}: {e}")

event_ids = pd.read_csv('event_ids.csv')['event_id'].tolist()
delete_all_events(service, event_ids)

Event with ID 3c77l3icqvgeofq549d8t5n778 deleted successfully.
Event with ID 8flk1hre7brb7fr1vint2nhfro deleted successfully.
Event with ID vmgrn57sacobtnktqu36sc7ud8 deleted successfully.
Event with ID 1u2668bj1qrjguo79cdat6s4pg deleted successfully.
Event with ID msgsudjq0uso82mu5t8pebcjqg deleted successfully.
Event with ID 10lqr72g1j7qu5nq95srk8mgd4 deleted successfully.
